In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import panel as pn
import param
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase

pn.extension()

# Set OpenAI key
os.environ["OPENAI_API_KEY"] = "sk-"  # Replace with your OpenAI key

# UI Widgets
file_input = pn.widgets.FileInput(accept=".csv")
query_input = pn.widgets.TextInput(placeholder="Ask a question...")
ask_button = pn.widgets.Button(name="Ask", button_type="primary")
clear_button = pn.widgets.Button(name="Clear", button_type="danger")
chat_box = pn.Column()

class CSVSQLChatbot(param.Parameterized):
    chat_history = param.List()
    agent = None
    df = None

    def __init__(self, **params):
        super().__init__(**params)
        self.chat_history = []

    def load_csv(self, event=None):
        if file_input.value is None:
            return pn.pane.Markdown("Please upload a CSV file.")
        with open("uploaded.csv", "wb") as f:
            f.write(file_input.value)
        self.df = pd.read_csv("uploaded.csv")
        self.init_agent()
        self.chat_history = []
        chat_box.clear()
        chat_box.append(pn.pane.Markdown("✅ CSV loaded. You can now ask questions."))

    def init_agent(self):
        engine = create_engine("sqlite:///:memory:")
        self.df.to_sql("aircraft", con=engine, index=False, if_exists="replace")
        db = SQLDatabase(engine)
        llm = ChatOpenAI(temperature=0, model="gpt-4")  # or gpt-3.5-turbo
        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        self.agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=False)

    def ask(self, event=None):
        if not self.agent:
            chat_box.append(pn.pane.Markdown("⚠️ Please load a CSV file first."))
            return
        user_query = query_input.value
        if not user_query:
            return
        chat_box.append(pn.Row("User:", pn.pane.Markdown(user_query)))
        try:
            answer = self.agent.run(user_query)
        except Exception as e:
            answer = f"Error: {e}"
        chat_box.append(pn.Row("Bot:", pn.pane.Markdown(answer)))
        query_input.value = ""

    def clear(self, event=None):
        self.chat_history = []
        chat_box.clear()

# Instantiate the chatbot
chatbot = CSVSQLChatbot()
file_input.param.watch(chatbot.load_csv, "value")
ask_button.on_click(chatbot.ask)
clear_button.on_click(chatbot.clear)

# Display UI
app_ui = pn.Column(
    pn.pane.Markdown("## 💬 CSV SQL Chatbot"),
    pn.Row(file_input),
    pn.Row(query_input, ask_button, clear_button),
    pn.Spacer(height=10),
    chat_box
)

app_ui.servable()


Column
    [0] Markdown(str)
    [1] Row
        [0] FileInput(accept='.csv')
    [2] Row
        [0] TextInput(placeholder='Ask a question...')
        [1] Button(button_type='primary', name='Ask')
        [2] Button(button_type='danger', name='Clear')
    [3] Spacer(height=10)
    [4] Column()